# Get all Tweets from a user and save it so a CSV File

### All you have to do is enter the Twitter's username in the variable bellow, and run the notebook

In [18]:
screen_name = 'tim_cook' #ominikbacher7'

In [2]:
import tweepy
import pandas as pd
from my_keys import *  # Import file with Twitter API access keys

In [19]:
def get_all_tweets(screen_name):
	auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
	auth.set_access_token(access_token, access_token_secret)

	api = tweepy.API(auth)

	all_tweets = []

	new_tweets = api.user_timeline(screen_name=screen_name, count=200)
	all_tweets.extend(new_tweets)
	
	# This will save the ID of the oldest tweet
	oldest = all_tweets[-1].id-1

	while len(new_tweets) > 0:
		#print (f"Fetching the tweets before {oldest} id")
		
		# Use the max_id param to prevent duplicates
		new_tweets = api.user_timeline(screen_name=screen_name, count=200, max_id=oldest)
		
		all_tweets.extend(new_tweets)
		
		# Update the oldest ID
		oldest = all_tweets[-1].id-1
		print (f"{len(all_tweets)} tweets have been fetched...")


	# Create empty dictionary
	all_tweets_dict = {}

	# Create some keys for the dictionary, with empty list for each value
	key_list = ['id', 'created_at', 'text', 'in_reply_to_status_id', 'in_reply_to_user_id', 'in_reply_to_screen_name',
		'geo', 'coordinates', 'place', 'contributors',
		'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'truncated']
	for key in key_list:
		all_tweets_dict[key] = []
	
	# List with the keys for the 'user' JSON key, which is a dictionary inside the main dictionary
	#    Then add the keys to the dictionary, again with empty list for each value
	user_list = ['id_str'] #, 'name', 'screen_name', 'followers_count', 'friends_count', 'listed_count', 'location', 'verified']
	for key in user_list:
		all_tweets_dict[key] = []
	

	# Fill the previous dictionary with the values from the JSON query
	for tweet in all_tweets:
		for key in key_list:
			all_tweets_dict[key].append(tweet._json[key])
		for key in user_list:
			all_tweets_dict[key].append(tweet._json['user'][key])
			
	# Change user id key so it's understood
	all_tweets_dict['user_id'] = all_tweets_dict.pop('id_str') # Change user id key to be better understandable

	df = pd.DataFrame.from_dict(all_tweets_dict)

	return all_tweets

In [20]:
tweets_df = get_all_tweets(screen_name)

400 tweets have been fetched...
600 tweets have been fetched...
800 tweets have been fetched...
1000 tweets have been fetched...
1200 tweets have been fetched...
1336 tweets have been fetched...
1336 tweets have been fetched...


In [21]:
tweets_df[2]._json

{'created_at': 'Thu May 19 01:17:10 +0000 2022',
 'id': 1527095988812255233,
 'id_str': '1527095988812255233',
 'text': 'Prehistoric Planet came roaring to life at a dino-mite Today at Apple session! We connected dinosaur fans with prod… https://t.co/ra4bZqPBfp',
 'truncated': True,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/ra4bZqPBfp',
    'expanded_url': 'https://twitter.com/i/web/status/1527095988812255233',
    'display_url': 'twitter.com/i/web/status/1…',
    'indices': [117, 140]}]},
 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1636590253,
  'id_str': '1636590253',
  'name': 'Tim Cook',
  'screen_name': 'tim_cook',
  'location': 'Cupertino',
  'description': "Apple CEO \uf8ff Auburn 🏀 🏈 Duke

In [27]:
tweets_df.to_csv(f'{screen_name}_tweets.csv')